# Import Aggregated IV Dataframe and join Medical Data

In [2]:
# import relevant tools
import os
from datetime import datetime

import numpy as np
import pandas as pd
from faker import Faker

fake = Faker()

# Import fire and PM2.5 data

In [3]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
!pip install geopandas --quiet
    from google.colab import drive

    drive.mount("/content/drive")
    in_dir = (
        in_instrument
    ) = "/content/drive/MyDrive/capstone_fire/notebooks/instrument_2/"
    import drive.MyDrive.capstone_fire.modules.deep_ols as deep_ols
else:
    in_dir = in_instrument = "../data/instrument_2/"
%cd '..'
    import modules.deep_ols as deep_ols
    %cd 'notebooks'

/Users/jordan/Documents/GitHub/fire_capstone
/Users/jordan/Documents/GitHub/fire_capstone/notebooks


In [44]:
# read in data that contains fire and pm25
fd = pd.read_csv(in_instrument + "finalish_df_6.csv")
suicide_df = pd.read_csv(in_instrument + "Suicide.csv")
neurotic_df = pd.read_csv(in_instrument + "Neurotic Disorders.csv")
psychoses_df = pd.read_csv(in_instrument + "Affective Psychoses.csv")

In [49]:
def prep_med(df):
    df["ZCTA"] = df["patzip_year_month"].map(lambda x: x[:5])
    df["year"] = df["patzip_year_month"].map(lambda x: x[6:10])
    df["month"] = df["patzip_year_month"].map(
        lambda x: "0" + str(x[-1]) if int(x[-2:]) < 10 else x[-2:]
    )
    df["year_month"] = df.apply(lambda x: str(x["year"]) + str(x["month"]), axis=1)
    df = df.rename(columns={"number_of_visits": str(df.columns[2]).lower()})
    df = df.drop(list(df.columns[:3]) + ["year", "month"], axis=1)
    return df  # [['number_of_visits','year','month','year_month']]


sf = prep_med(suicide_df)
nf = prep_med(neurotic_df)
pf = prep_med(psychoses_df)

fd.ZCTA = fd.ZCTA.astype(str)
fd.year_month = fd.year_month.astype(str)

fd1 = fd.merge(sf, on=["ZCTA", "year_month"], how='left')
fd2 = fd1.merge(nf, on=["ZCTA", "year_month"], how='left')
df = fd2.merge(pf, on=["ZCTA", "year_month"], how='left')
df = df.drop_duplicates()

In [50]:
df.to_csv(in_instrument + "final_df_3.csv")